<a href="https://colab.research.google.com/github/knkarthick/bart-large-xsum-samsum-dataset/blob/main/Copy_of_Fine_Tune_bart_large_xsum_for_Dialogue_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed May 26 18:33:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install datasets transformers rouge-score nltk py7zr
! pip install wandb -qq

     |████████████████████████████████| 225kB 8.3MB/s 
     |████████████████████████████████| 2.3MB 48.0MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 245kB 47.2MB/s 
     |████████████████████████████████| 112kB 49.8MB/s 
     |████████████████████████████████| 901kB 31.9MB/s 
     |████████████████████████████████| 3.3MB 47.6MB/s 
     |████████████████████████████████| 358kB 36.2MB/s 
     |████████████████████████████████| 1.9MB 48.3MB/s 
     |████████████████████████████████| 2.2MB 31.6MB/s 
     |████████████████████████████████| 122kB 56.8MB/s 
ERROR: transformers 4.6.1 has requirement huggingface-hub==0.0.8, but you'll have huggingface-hub 0.0.9 which is incompatible.
     |████████████████████████████████| 1.8MB 8.0MB/s 
     |████████████████████████████████| 102kB 13.7MB/s 
     |████████████████████████████████| 174kB 39.2MB/s 
     |████████████████████████████████| 133kB 36.4MB/s 
     |███████████████████████████████

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
%env WANDB_PROJECT=bart_large_xsum_samsum

env: WANDB_PROJECT=bart_large_xsum_samsum


In [ ]:
model_checkpoint = "facebook/bart-large-xsum"

## Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("samsum")
metric = load_metric("rouge")

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


## Preprocessing the data

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    "test-dialogue-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",  # enable logging to W&B
    run_name="bart-large-xsum-samsum"  # name of the W&B run (optional)
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
trainer.train()

wandb: Currently logged in as: lidiya (use `wandb login --relogin` to force relogin)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.384400,1.481299,53.114800,28.340200,43.631000,48.439400,26.657700
1,1.045900,1.428414,54.377800,29.504400,44.564400,49.751000,28.954800
2,0.855600,1.478123,54.392100,29.807800,45.154300,49.942000,30.028100


TrainOutput(global_step=5523, training_loss=1.1156969921713653, metrics={'train_runtime': 7266.8301, 'train_samples_per_second': 0.76, 'total_flos': 144216851742720.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 961540096, 'init_mem_gpu_alloc_delta': 1625362944, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -1326395392, 'train_mem_gpu_alloc_delta': 4891765248, 'train_mem_cpu_peaked_delta': 1353306112, 'train_mem_gpu_peaked_delta': 5291012608})

In [ ]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_gen_len': 30.0281,
 'eval_loss': 1.478122591972351,
 'eval_mem_cpu_alloc_delta': 303104,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 1703727616,
 'eval_rouge1': 54.3921,
 'eval_rouge2': 29.8078,
 'eval_rougeL': 45.1543,
 'eval_rougeLsum': 49.942,
 'eval_runtime': 504.7343,
 'eval_samples_per_second': 1.621}

In [ ]:
wandb.finish()

train/loss,0.8556
train/learning_rate,0.0
train/epoch,3.0
train/global_step,5523
_runtime,7772
_timestamp,1622061938
_step,15
eval/loss,1.47812
eval/rouge1,54.3921
eval/rouge2,29.8078
eval/rougeL,45.1543


train/loss,█▇▆▅▃▃▃▂▁▁▁
train/learning_rate,█▇▇▆▅▄▄▃▂▂▁
train/epoch,▁▂▂▃▃▄▄▅▅▆▇▇████
train/global_step,▁▂▂▃▃▄▄▅▅▆▇▇████
_runtime,▁▂▂▃▃▃▄▄▅▅▆▆▇███
_timestamp,▁▂▂▃▃▃▄▄▅▅▆▆▇███
_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
eval/loss,█▁██
eval/rouge1,▁███
eval/rouge2,▁▇██
eval/rougeL,▁▅██


[Uploaded the model](https://huggingface.co/transformers/model_sharing.html) to the [🤗 Model Hub](https://huggingface.co/models). You can use it to generate results as shown below.

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="lidiya/bart-large-xsum-samsum")
conversation = '''Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye                                       
'''
summarizer(conversation)

[{'summary_text': "Amanda can't find Betty's number. Larry called her last time they were at the park. Hannah doesn't know him very well. Amanda suggests Hannah to text him."}]